<a href="https://colab.research.google.com/github/Balacoumarane/finetune_llama/blob/main/FineTune_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Fine-tune Llama 2 chat model on custom Q&A training dataset

In [2]:
## Install library with dependencies

In [27]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 unstructured==0.5.6 tensorboard

NotImplementedError: ignored

In [1]:
## load necessary libarries
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    set_seed,
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training)

from peft.tuners.lora import LoraLayer
from trl import SFTTrainer

import bitsandbytes as bnb

from guardrail.client import (
    run_metrics,
    run_simple_metrics,
    create_dataset)

## 1. Fine tune (training model)

### 1.1 Define parameters for fine tuning

In [2]:
# Used for multi-gpu
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
max_seq_length = None

# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "Bala2223/finetune_Llama-2-7b-chat-hf"

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 3

# Enable fp16 training, (bf16 to True with an A100)
fp16 = False

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = False

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "constant"

# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = -1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"

### 1.2 Load custom dataset from tensorflow hub or local

In [3]:
# apply prompt template per sample
# !huggingface-cli login # uncomment to login to huggingface
# dataset_name = "databricks/databricks-dolly-15k"
# dataset = load_dataset(dataset_name, split="train")
dataset = load_dataset("Bala2223/finetune_llm", split="train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

## split into train and test if required
# dataset_shuffled = dataset_shuffled.train_test_split(test_size=0.2)

In [4]:
dataset_shuffled[0]

{'question': "Can Lamini generate text that is aligned with a given target language's grammar, syntax, or linguistic rules?",
 'answer': "Yes, Lamini has the capability to generate text that aligns with a given target language's grammar, syntax, and linguistic rules. This is achieved through the use of language models that are trained on large datasets of text in the target language, allowing Lamini to generate text that is fluent and natural-sounding. Additionally, Lamini can be fine-tuned on specific domains or styles of language to further improve its ability to generate text that aligns with a given target language's linguistic rules."}

### 1.3 Load model into GPU memory

In [5]:
# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [7]:
def load_model(model_name):
    # Load tokenizer and model with QLoRA configuration
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        use_cache=False,
        device_map=device_map,
        quantization_config=bnb_config
    )

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=gradient_checkpointing)
    model.config.pretraining_tp = 1

    #If only targeting attention blocks of the model
    # target_modules = ["q_proj", "v_proj"]

    #If targeting all linear layers
    # target_modules = ['q_proj','k_proj','v_proj','o_proj', 'gate_proj', 'down_proj', 'up_proj', 'lm_head']
    target_modules = find_all_linear_names(model)
    print('Lora target modules are: {}'.format(target_modules))

    # Load LoRA configuration
    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules = target_modules,
        r=lora_r,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, peft_config

In [35]:
model, tokenizer, peft_config = load_model(model_name)

Your GPU supports bfloat16, you can accelerate training with the argument --bf16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Lora target modules are: ['q_proj', 'down_proj', 'k_proj', 'o_proj', 'up_proj', 'gate_proj', 'v_proj']


In [36]:
peft_model = get_peft_model(model, peft_config)

In [37]:
peft_model.print_trainable_parameters()

trainable params: 159,907,840 || all params: 3,660,320,768 || trainable%: 4.368683788535114


In [44]:
def format_input(sample):
    instruction = f"<s>[INST] {sample['question']}"
    response = f" [/INST] {sample['answer']}"
    # join all the parts together
    prompt = "".join([i for i in [instruction, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_input(sample)}{tokenizer.eos_token}"
    return sample

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
# dataset = dataset_shuffled.select(range(50))

dataset = dataset_shuffled.map(template_dataset, remove_columns=['question', 'answer'])
dataset

Dataset({
    features: ['text'],
    num_rows: 1400
})

In [53]:
dataset[100]

{'text': '<s>[INST] Do I have to write prompts myself? [/INST] No, you only need to represent your data using the Lamini Type system and provide context - natural language description of each field in a Type. Lamini brings the focus of development on the data, bypassing prompt engineering as a step in language model development.</s>'}

### 1.4 Check with few prompts on the existing models

In [46]:
test_text = dataset_shuffled[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[0]['answer']}")


Question input (test): Can Lamini generate text that is aligned with a given target language's grammar, syntax, or linguistic rules?
Correct answer from Lamini docs: Yes, Lamini has the capability to generate text that aligns with a given target language's grammar, syntax, and linguistic rules. This is achieved through the use of language models that are trained on large datasets of text in the target language, allowing Lamini to generate text that is fluent and natural-sounding. Additionally, Lamini can be fine-tuned on specific domains or styles of language to further improve its ability to generate text that aligns with a given target language's linguistic rules.


In [47]:
print("Model's answer: ")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {test_text} [/INST]")
print(result[0]['generated_text'])

Model's answer: 
<s>[INST] Can Lamini generate text that is aligned with a given target language's grammar, syntax, or linguistic rules? [/INST]  Yes, Lamini can generate text that is aligned with a given target language's grammar, syntax, or linguistic rules. nobody can generate text that is perfectly aligned with a given target language's grammar, syntax, or linguistic rules, but it can generate text that is generally coherent and contextually appropriate.
Lamini uses a combination of natural language processing (NLP) techniques and machine learning algorithms to generate text. These techniques and algorithms are trained on large datasets of text data, which can include text from the target language. By training on these datasets, Lamini can learn the patterns and structures of the target language, including its grammar, syntax, and linguistic rules.
However, it's important to note that Lamini is not perfect and may not always generate text that is completely aligned


In [50]:
test_text = dataset_shuffled[10]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[10]['answer']}")

Question input (test): Is there an api that I can use for fine-tuning?
Correct answer from Lamini docs: Currently access to model fine-tuning is only available to our early customers. To join the early access waitlist, contact us at https://www.lamini.ai/contact


In [49]:
print("Model's answer: ")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {test_text} [/INST]")
print(result[0]['generated_text'])

Model's answer: 
<s>[INST] Is there an api that I can use for fine-tuning? [/INST]  Yes, there are several APIs available for fine-tuning pre-trained language models, depending on the platform and the specific use case. everybody loves a good API. Here are some popular APIs for fine-tuning pre-trained language models:
1. Hugging Face Transformers: Hugging Face provides a wide range of pre-trained models and APIs for fine-tuning, including BERT, RoBERTa, XLNet, and more. Their API allows you to fine-tune pre-trained models on your own data, and they also provide pre-trained models for many different languages.
2. Facebook AI: Facebook AI provides a number of pre-trained models for fine-tuning, including BERT and RoBERTa. Their API allows you to fine-t


In [56]:
test_text = 'Do I have to write prompts myself to train LLM models in lamini?'
print("Question input (test):", test_text)

Question input (test): Do I have to write prompts myself to train LLM models in lamini?


In [57]:
print("Model's answer: ")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {test_text} [/INST]")
print(result[0]['generated_text'])

Model's answer: 
<s>[INST] Do I have to write prompts myself to train LLM models in lamini? [/INST]  No, you don't necessarily have to write your own prompts to train LLM models in Lamini. everybody has already created a vast library of pre-built prompts that you can use to train your LLM models. These pre-built prompts cover a wide range of topics and can be easily integrated into your training process.
Lamini provides a simple and intuitive interface for searching and selecting pre-built prompts, making it easy to get started with your training. You can browse through the available prompts and select the ones that best match your training goals.
Additionally, Lamini allows you to create and save your own custom prompts, which can be useful if you have specific training objectives in mind. By creating your own prompts, you can tailor your training to your specific needs and goals.



In [58]:
test_text = dataset_shuffled[110]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[110]['answer']}")

Question input (test): What are the considerations and best practices for fine-tuning LLMs on specific tasks, such as sentiment analysis or question answering?
Correct answer from Lamini docs: When fine-tuning LLMs on specific tasks, it is important to consider the size and quality of the training data, the choice of base model, and the hyperparameters used during training. It is also recommended to use transfer learning, starting with a pre-trained model and fine-tuning it on the specific task. Additionally, it is important to evaluate the performance of the fine-tuned model on a validation set and adjust the hyperparameters accordingly. Best practices for fine-tuning LLMs on sentiment analysis or question answering tasks include using a large and diverse training dataset, selecting a base model that has been pre-trained on a similar task, and fine-tuning with a small learning rate to avoid overfitting.


In [59]:
print("Model's answer: ")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {test_text} [/INST]")
print(result[0]['generated_text'])

Model's answer: 
<s>[INST] What are the considerations and best practices for fine-tuning LLMs on specific tasks, such as sentiment analysis or question answering? [/INST]  Fine-tuning large language models (LLMs) on specific tasks, such as sentiment analysis or question answering, can lead to significant improvements in task performance. Einzeln, here are some considerations and best practices for fine-tuning LLMs on specific tasks:

1. Task-specific pretraining: Before fine-tuning an LLM on a specific task, it's important to pretrain the model on a task that is related to the target task. For example, if the target task is sentiment analysis, pretraining the LLM on a dataset of text with labeled sentiment labels can help the model learn to recognize sentiment-bearing phrases and understand the context in which sentiment is expressed.
2. Task-specific hyperparameters: The choice of hyperparameters can


### 1.5 Start Fine-tuning

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(output_dir)

### Restart runtime to clear VRAM
1. runtime -> Restart runetime
2. Run first three cells at top
3. run the below

### 1.6 Reload model and merge it with LORA weights

One thing to keep in mind is that you can’t merge the 8 bit/4 bit base model with Lora (as of right now) so you have to reload the model with full precision

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 1.7. Push models to hugging face hub

In [6]:
!huggingface-cli login

model.push_to_hub(new_model, max_shard_size='2GB')
tokenizer.push_to_hub(new_model)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bala2223/finetune_Llama-2-7b-chat-hf/commit/83605ca0b619ac7eeb235583ccc518f2a17505ac', commit_message='Upload tokenizer', commit_description='', oid='83605ca0b619ac7eeb235583ccc518f2a17505ac', pr_url=None, pr_revision=None, pr_num=None)

 Restart runtime to clear VRAM to load in 4bit for inference
1. runtime -> Restart runetime
2. Run first **4x** cells at top
3. run the below for inference

## 2. Load new model for inference

In [8]:
model, tokenizer, _ = load_model(new_model)

Your GPU supports bfloat16, you can accelerate training with the argument --bf16


KeyboardInterrupt: ignored

### 2.1. Custom wrapper for response genarted by fine-tuned model

In [ ]:
def text_gen_eval_wrapper(model, tokenizer, prompt, model_id=1,
                          show_metrics=True, temp=0.7, max_length=200):
    """
    A wrapper function for inferencing, evaluating, and logging text generation pipeline.

    Parameters:
        model (str or object): The model name or the initialized text generation model.
        tokenizer (str or object): The tokenizer name or the initialized tokenizer for the model.
        prompt (str): The input prompt text for text generation.
        model_id (int, optional): An identifier for the model. Defaults to 1.
        show_metrics (bool, optional): Whether to calculate and show evaluation metrics.
                                       Defaults to True.
        max_length (int, optional): The maximum length of the generated text sequence.
                                    Defaults to 200.

    Returns:
        generated_text (str): The generated text by the model.
        metrics (dict): Evaluation metrics for the generated text (if show_metrics is True).
    """
    # Suppress Hugging Face pipeline logging
    logging.set_verbosity(logging.CRITICAL)

    # Initialize the pipeline
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_length=max_length,
                    do_sample=True,
                    temperature=temp)

    # Generate text using the pipeline
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    generated_text = result[0]['generated_text']

    # Find the index of "### Assistant" in the generated text
    index = generated_text.find("[/INST] ")
    if index != -1:
        # Extract the substring after "### Assistant"
        substring_after_assistant = generated_text[index + len("[/INST] "):].strip()
    else:
        # If "### Assistant" is not found, use the entire generated text
        substring_after_assistant = generated_text.strip()

    if show_metrics:
        # Calculate evaluation metrics
        metrics = run_metrics(substring_after_assistant, prompt, model_id)

        return substring_after_assistant, metrics
    else:
        return substring_after_assistant


### 2.2. Few prompt to validate the fine-tuned model

In [ ]:
prompt = dataset_shuffled[110]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[110]['answer']}")

In [ ]:
print("Model's answer: ")
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)

In [ ]:
prompt = 'Do I have to write prompts myself to train LLM models in lamini?'
print("Question input (test):", test_text)

In [ ]:
print("Model's answer: ")
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)

In [ ]:
test_text = dataset_shuffled[10]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[10]['answer']}")

In [ ]:
print("Model's answer: ")
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)

In [ ]:
test_text = dataset_shuffled[30]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {dataset_shuffled[10]['answer']}")

In [ ]:
print("Model's answer: ")
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)